In [1]:
import sys
sys.path.insert(0, '../nn-compression')

from slender.coding import Codec
import os
import argparse
import time

import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data.distributed
import torch.nn as nn
from src_files.helper_functions.bn_fusion import fuse_bn_recursively
from src_files.models import create_model
import matplotlib

from src_files.models.tresnet.tresnet import InplacABN_to_ABN
from dataset import MultiLabelDataset

matplotlib.use('TkAgg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm

/home/administrator/anaconda3/envs/5329/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# parser = argparse.ArgumentParser(description='PyTorch MS_COCO infer')
# parser.add_argument('--num-classes', default=80, type=int)
# parser.add_argument('--model-path', type=str, default='tresnet_xl_COCO_640_91_4.pth')
# parser.add_argument('--pic-path', type=str, default='./pics/000000000885.jpg')
# parser.add_argument('--model-name', type=str, default='tresnet_xl')
# parser.add_argument('--image-size', type=int, default=640)
# # parser.add_argument('--dataset-type', type=str, default='MS-COCO')
# parser.add_argument('--th', type=float, default=0.75)
# parser.add_argument('--top-k', type=float, default=20)
# # ML-Decoder
# parser.add_argument('--use-ml-decoder', default=1, type=int)
# parser.add_argument('--num-of-groups', default=-1, type=int)  # full-decoding
# parser.add_argument('--decoder-embedding', default=768, type=int)
# parser.add_argument('--zsl', default=0, type=int)

class Args:
  num_classes = 80
  model_path = 'tresnet_xl_COCO_640_91_4.pth'
  model_name = 'tresnet_xl'
  image_size = 640
  use_ml_decoder = 1
  num_of_groups = -1
  decoder_embedding = 768
  zsl = 0
  
args = Args()

In [4]:
backend = "fbgemm"

# Setup model
print('creating model {}...'.format('tresnet_xl'))
model = create_model(args, load_head=True).cuda()
state = torch.load(args.model_path, map_location='cpu')
model.load_state_dict(state['model'], strict=True)
########### eliminate BN for faster inference ###########
model = model.cpu()
model = InplacABN_to_ABN(model)
model = fuse_bn_recursively(model)
model = model.half().eval()
#model = model.fl#oat().eval()
rule = []
for m, v in model.named_parameters():
    if m[-6:] == 'weight':
        rule.append((m, 'huffman', 0, 0, 4))
print(len(rule))

creating model tresnet_xl...
254


In [ ]:
codec = Codec(rule=rule)
encoded_model = codec.encode(model = model)